# General Structure

## test vs training
* is it even possible to predict a test set with this little data? We have notes and chords as guidance for what the test set should be predicted on. If we don't see those notes or chords elsewhere, do we have any chance of succeeding?
* or should we just say fuck it, and try, and if we can make it reasonably far we'll add a few more training inputs (from either Mingus or some other band playing the real book. That way we know how the test set sound could sound)

## inputs
* amplitude vs time or frequency (what kind of FFT?) vs time
* real book vs time
    * notes on treble, chords on bass?
    * how closely do we have to map it to the song (I think pretty fucking closely)
    * how much data per note/chord (there are modifiers, pauses, etc.)
    * the Magenta Note Sequence actually looks pretty good for this -- why not use their helper functions to create sequences of notes and chords for input? 
* is the real book the only input + memory? And we're learning the memory?

## model RNN
* notice delta compared to the real book -- is that delta consistent?
* Does it make sense to build a GAN that goes from note input to sound output, and one that goes from sound output to not input and try to make the discriminator identify if it's real or not? Yes!
* Is there an advantage to using a CNN of the spectograph? All d
* forgetting or not
* bidrectional?
* what is the memory? That feels like how the band modifies the notes -- (how we translate the real book to the wave form -- modifications and global sound). 
* RNN structure:
    * output of previous step is the amplitude played, then should be input into the next step, along with real book.
    * all a's from previous step should be passed along
    * DanQ structure (cnn -> rnn across a reduced space -> fully connected). Rnns keep parameters small, but blow up gradient calculations. 

### Measures. Format is: note, how long

* #### Pre
    * C (low) 1/8
* #### 1
    * notes
    * F 1/8, A 1/8, A 1/12, F 1/12, A 1/12, B 1/8, A 1/4, F 1/16, D 1/16
    * chords
    * F7#9 (F - A - C - Eb - G#)
    * Db7 (Db - F - Ab - B)
   
* #### 2
    * notes
    * F 1/8, A 1/4, F 1/16, E 1/16, F 3/8, C 1/8
    * chords
    * GbMaj7 (Gb - Bb - Db - F)
    * B#11 7 (B - A - D# - E#)???
* #### 3
    * notes
    * F 1/8, A 1/8, A 1/12, F 1/12, A 1/12, Cb 1/8, B 1/4, F 1/16, E 1/16
    * chords
    * Eb7sus4 (Eb - Ab - Bb - Db) 
    * Db7 (Db - F - Ab - B)
* #### 4
    * notes
    * F 1/8, A 1/4, F 1/16, E 1/16, F 3/8, F 1/8
    * chords
    * Eb7sus4 (Eb - Ab - Bb - Db)
    * F7 (F - A - C - Eb)
* #### 5
    * notes
    * C 1/8, E (octave up) 1/8, E (octave up) 1/12, F 1/12, A 1/12, B 1/8, A 1/4, F 1/16, A 1/16
    * chords
    * Bbm7 (Bb - Db - F - Ab)
    * Db7 (Db - F - Ab - B)
* #### 6
    * notes
    * C 1/8, F (octave up) 1/8, F (octave up) 1/12, F 1/12, B 1/12, E (octave up) 2/12, C 1/12,  A 1/12, E# 1/12, D (low) 1/12
    * chords
    * Gm7 G - Bb - D - F
    * C7#5 C - E - G# - Bb
* #### 7 ?
    * notes
    * A 1/6, C flat 1/6, F 1/6, E sharp 1/6, C 1/6, G 1/6
    * chords
    * D7 D - F# - A - C
    * G7  G - B - D - F
* #### 8
    * notes
    * A 1/2, F 3/8, Cb 1/16, B 1/16
    * chords
    * Db7 (Db - F - Ab - B)
    * GbMaj7 (Gb - Bb - Db - F)
* #### 9
    * notes
    *  A 1/2, F 1/2
    * chords
    * B^7_6 ?
    * Bb7 Bb - D - F - Ab
* #### 10
    * notes
    * B 1/8, A 1/8, F 1/8, E 1/8, Cb 1/8, B 1/8, A 1/8, F 1/8
    * chords
    * C7 C - E - G - Bb
    * Eb7 Eb - G - Bb - Db
* #### 11
    * notes
    * A 1/2, F 1/2
    * chords
    * F7#9 (F - A - C - Eb - G#)
    * Db7 (Db - F - Ab - B)
* #### 12
    * notes
    * C 1/6, B 1/6, A 1/6, F 1/6, E 1/6, C (low) 1/6
    * chords
    * GbMaj#11 G Bb C D Gb (Gb Bb B Db F?)
    * B7b5 B Eb F A

In [ ]:
from utilities import array_statistics, read_wav, append_array
import IPython.display as ipd
import librosa, librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os



GPPH_DATA_DIRECTORY = '/Users/pbatra/projects/lil_wayne/data/10_17_2018'

gpph_files = [f for f in os.listdir(GPPH_DATA_DIRECTORY) if os.path.isfile(os.path.join(GPPH_DATA_DIRECTORY, f))]
print gpph_files

for file_ in gpph_files[:1]:
    filename = os.path.join(GPPH_DATA_DIRECTORY, file_)
    x, sr = read_wav(filename)
    
    

In [ ]:
def sample_array_dumbly(x, window):
    total_steps = len(x)
    y = np.zeros((total_steps,))
    for idx in range(total_steps/window):
        y[(idx * window):((idx+1) * window)] = np.mean(x[(idx*window): ((idx+1) * window)])
    return y

def sample_array_tft(x, window):
    array_statistics(x, 44100)
    tft_x = librosa.stft(x)
    mean_values = np.mean(abs(tft_x), axis = 0)
    print np.count_nonzero(tft_x[abs(tft_x) > 0])
    tft_x[(abs(tft_x) - window * mean_values) < 0] = 0
    print np.count_nonzero(tft_x[abs(tft_x) > 0])
    itft_tft_x = librosa.istft(tft_x)
    array_statistics(itft_tft_x, 44100)
    return itft_tft_x

In [ ]:
print x.shape, sr
#print filepath
#ipd.Audio(filepath)
ipd.Audio(sample_array_tft(x,1.0), rate = sr)

In [ ]:
tft_x = librosa.stft(x, 2048)
tft_x.shape

In [ ]:
mean_per_slice = np.mean(abs(tft_x), axis=0)


In [ ]:
blah = np.random.rand(3,10)
mean_per_slice = np.mean(abs(blah), axis=0)
print blah
print mean_per_slice
blah[(blah-mean_per_slice) < 0] = 0
print blah


In [ ]:
len(x)